# My Code For Frequent Pattern Growth

In [5]:
# Create node class
class Node:
    def __init__(self, item, frequency, parent):
        self.item = item
        self.frequency = frequency
        self.parent = parent
        self.children = {}
        self.next = None

# Create header table
def create_header_table(dataset, min_support):
    header_table = {}
    for basket in dataset:
        for item in basket:
            header_table[item] = header_table.get(item, 0) + 1
    header_table = {k: v for k, v in header_table.items() if v >= min_support}
    return header_table

# update tree
def update_tree(item, node, header_table):
    if item in node.children:
        node.children[item].frequency += 1
    else:
        new_node = Node(item, 1, node)
        node.children[item] = new_node

        if header_table.get(item) is None:
            header_table[item] = [0, None]

        if header_table[item][1] is None:
            header_table[item][1] = new_node
        else:
            update_linked_list(header_table[item][1], new_node)

    return node.children[item]

#update linked list
def update_linked_list(head, node):
    while head.next is not None:
        head = head.next
    head.next = node

# Main fp_growth function
def fp_growth(dataset, min_support):
    header_table = create_header_table(dataset, min_support)
    if len(header_table) == 0:
        return None

    for item in header_table:
        header_table[item] = [header_table[item], None]

    root = Node('Null', 1, None)
    for basket in dataset:
        sorted_items = sorted(basket, key=lambda k: header_table.get(k, [0])[0], reverse=True)
        current_node = root
        for item in sorted_items:
            current_node = update_tree(item, current_node, header_table)

    frequent_patterns = []
    for item in header_table:
        base_pattern = []
        conditional_tree = []
        node = header_table[item][1]

        while node is not None:
            conditional_tree.append(node)
            node = node.parent

        for ct_node in conditional_tree:
            base_pattern.append(ct_node.item)

        if len(base_pattern) > 1:
            frequent_patterns.append(base_pattern)

    return frequent_patterns




In [6]:
# Example usage:
# Sample dataset (list of baskets with fruits)
dataset = [
    ['apple', 'banana', 'cherry'],
    ['apple', 'banana', 'dates'],
    ['banana', 'cherry', 'elderberry'],
    ['apple', 'cherry', 'dates', 'elderberry'],
    ['apple', 'dates', 'elderberry']
]

# Minimum support count
min_support = 2

frequent_itemsets = fp_growth(dataset, min_support)
print("Frequent Itemsets:", frequent_itemsets)

Frequent Itemsets: [['apple', 'Null'], ['banana', 'apple', 'Null'], ['cherry', 'banana', 'apple', 'Null'], ['dates', 'banana', 'apple', 'Null'], ['elderberry', 'cherry', 'banana', 'Null']]


## Modern Library MLXTEND for FPG tests
#### Imports

#### Code Inspiration: https://www.youtube.com/watch?v=Cryve9ZWbYk&t=161s


In [1]:
import pandas as pd
import random 
import numpy as np
!pip install mlxtend
from mlxtend.frequent_patterns import fpgrowth
from mlxtend.frequent_patterns import association_rules
from mlxtend.preprocessing import TransactionEncoder

error: externally-managed-environment

× This environment is externally managed
╰─> To install Python packages system-wide, try 'pacman -S
    python-xyz', where xyz is the package you are trying to
    install.
    
    If you wish to install a non-Arch-packaged Python package,
    create a virtual environment using 'python -m venv path/to/venv'.
    Then use path/to/venv/bin/python and path/to/venv/bin/pip.
    
    If you wish to install a non-Arch packaged Python application,
    it may be easiest to use 'pipx install xyz', which will manage a
    virtual environment for you. Make sure you have python-pipx
    installed via pacman.

note: If you believe this is a mistake, please contact your Python installation or OS distribution provider. You can override this, at the risk of breaking your Python installation or OS, by passing --break-system-packages.
hint: See PEP 668 for the detailed specification.


ModuleNotFoundError: No module named 'mlxtend'

#### Make random dataset

In [18]:
groceries = ['Apples', 'Bananas', 'Milk', 'Bread', 'Eggs', 'Cheese', 'Tomatoes', 'Potatoes', 'Onions', 'Chicken']

# Generating 6 arrays with shuffled orders and 70% chance for 1, 30% chance for 0 for each item
arrays = []
for _ in range(6):
    random.shuffle(groceries)  # Shuffle the order of groceries
    random_frequency = [random.choices([0, 1], weights=[0.3, 0.7])[0] for _ in range(len(groceries))]
    arrays.append(dict(zip(groceries, random_frequency)))

# Displaying the generated arrays
for index, arr in enumerate(arrays, 1):
    print(f"Array {index}: {arr}")



Array 1: {'Potatoes': 1, 'Bread': 1, 'Tomatoes': 1, 'Onions': 1, 'Milk': 1, 'Eggs': 1, 'Bananas': 1, 'Chicken': 1, 'Cheese': 1, 'Apples': 1}
Array 2: {'Apples': 1, 'Bananas': 0, 'Onions': 1, 'Tomatoes': 1, 'Potatoes': 1, 'Milk': 1, 'Chicken': 1, 'Eggs': 1, 'Bread': 1, 'Cheese': 0}
Array 3: {'Bananas': 1, 'Eggs': 1, 'Bread': 1, 'Apples': 1, 'Chicken': 1, 'Onions': 1, 'Tomatoes': 1, 'Cheese': 1, 'Milk': 1, 'Potatoes': 1}
Array 4: {'Cheese': 0, 'Eggs': 0, 'Bananas': 1, 'Milk': 1, 'Chicken': 1, 'Bread': 1, 'Apples': 1, 'Potatoes': 1, 'Tomatoes': 1, 'Onions': 0}
Array 5: {'Apples': 1, 'Milk': 1, 'Cheese': 1, 'Tomatoes': 1, 'Potatoes': 1, 'Chicken': 1, 'Bananas': 1, 'Onions': 1, 'Eggs': 1, 'Bread': 1}
Array 6: {'Bread': 1, 'Eggs': 1, 'Cheese': 1, 'Milk': 1, 'Potatoes': 1, 'Apples': 1, 'Bananas': 1, 'Tomatoes': 0, 'Onions': 1, 'Chicken': 1}


#### Clean up dataset

In [19]:
dataset = [
    ['Milk', 'Cheese', 'Eggs', 'Potatoes', 'Bananas', 'Onions'],
    ['Eggs', 'Bread', 'Cheese', 'Potatoes', 'Milk', 'Bananas'],
    ['Potatoes', 'Bread', 'Milk', 'Onions', 'Cheese', 'Chicken', 'Tomatoes'],
    ['Bananas', 'Bread', 'Apples', 'Onions', 'Potatoes', 'Tomatoes'],
    ['Potatoes', 'Tomatoes', 'Onions', 'Apples', 'Eggs', 'Cheese', 'Bread'],
    ['Chicken', 'Potatoes', 'Apples', 'Bread', 'Cheese', 'Milk']
]

print(dataset)

[['Milk', 'Cheese', 'Eggs', 'Potatoes', 'Bananas', 'Onions'], ['Eggs', 'Bread', 'Cheese', 'Potatoes', 'Milk', 'Bananas'], ['Potatoes', 'Bread', 'Milk', 'Onions', 'Cheese', 'Chicken', 'Tomatoes'], ['Bananas', 'Bread', 'Apples', 'Onions', 'Potatoes', 'Tomatoes'], ['Potatoes', 'Tomatoes', 'Onions', 'Apples', 'Eggs', 'Cheese', 'Bread'], ['Chicken', 'Potatoes', 'Apples', 'Bread', 'Cheese', 'Milk']]


#### Convert the dataset to true or false values

In [20]:
te = TransactionEncoder()
te_array = te.fit(dataset).transform(dataset)
te_array

array([[False,  True, False,  True, False,  True,  True,  True,  True,
        False],
       [False,  True,  True,  True, False,  True,  True, False,  True,
        False],
       [False, False,  True,  True,  True, False,  True,  True,  True,
         True],
       [ True,  True,  True, False, False, False, False,  True,  True,
         True],
       [ True, False,  True,  True, False,  True, False,  True,  True,
         True],
       [ True, False,  True,  True,  True, False,  True, False,  True,
        False]])

#### Convert to dataframe

In [21]:
df = pd.DataFrame(te_array, columns = te.columns_)
df

,Apples,Bananas,Bread,Cheese,Chicken,Eggs,Milk,Onions,Potatoes,Tomatoes
0,False,True,False,True,False,True,True,True,True,False
1,False,True,True,True,False,True,True,False,True,False
2,False,False,True,True,True,False,True,True,True,True
3,True,True,True,False,False,False,False,True,True,True
4,True,False,True,True,False,True,False,True,True,True
5,True,False,True,True,True,False,True,False,True,False


## FPG Algorith

In [24]:
fpgrowth(df, min_support=0.6)

,support,itemsets
0,1.000000,(8)
1,0.833333,(3)
2,0.666667,(7)
3,0.666667,(6)
4,0.833333,(2)
5,0.833333,"(8, 3)"
6,0.666667,"(8, 7)"
7,0.666667,"(3, 6)"
8,0.666667,"(8, 6)"
9,0.666667,"(8, 3, 6)"


## Apriori Algo

In [22]:
from mlxtend.frequent_patterns import apriori
frequent_itemsets=apriori(df, min_support=0.6, use_colnames=True)
frequent_itemsets

,support,itemsets
0,0.833333,(Bread)
1,0.833333,(Cheese)
2,0.666667,(Milk)
3,0.666667,(Onions)
4,1.000000,(Potatoes)
5,0.666667,"(Bread, Cheese)"
6,0.833333,"(Potatoes, Bread)"
7,0.666667,"(Cheese, Milk)"
8,0.833333,"(Potatoes, Cheese)"
9,0.666667,"(Potatoes, Milk)"


In [23]:
rules = association_rules(frequent_itemsets, metric = "confidence", min_threshold = 0.8)
rules

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
0,(Potatoes),(Bread),1.000000,0.833333,0.833333,0.833333,1.0,0.000000,1.0,0.0
1,(Bread),(Potatoes),0.833333,1.000000,0.833333,1.000000,1.0,0.000000,inf,0.0
2,(Milk),(Cheese),0.666667,0.833333,0.666667,1.000000,1.2,0.111111,inf,0.5
3,(Potatoes),(Cheese),1.000000,0.833333,0.833333,0.833333,1.0,0.000000,1.0,0.0
4,(Cheese),(Potatoes),0.833333,1.000000,0.833333,1.000000,1.0,0.000000,inf,0.0
5,(Milk),(Potatoes),0.666667,1.000000,0.666667,1.000000,1.0,0.000000,inf,0.0
6,(Onions),(Potatoes),0.666667,1.000000,0.666667,1.000000,1.0,0.000000,inf,0.0
7,"(Bread, Cheese)",(Potatoes),0.666667,1.000000,0.666667,1.000000,1.0,0.000000,inf,0.0
8,"(Potatoes, Milk)",(Cheese),0.666667,0.833333,0.666667,1.000000,1.2,0.111111,inf,0.5
9,"(Cheese, Milk)",(Potatoes),0.666667,1.000000,0.666667,1.000000,1.0,0.000000,inf,0.0


In [25]:
from mlxtend.frequent_patterns import apriori
%timeit apriori(df, min_support = 0.6)

1.56 ms ± 84.6 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [26]:
%timeit fpgrowth(df, min_support = 0.6)

534 µs ± 12.6 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
